Подключаем библиотеки

In [172]:
from nornir import InitNornir
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
from nornir_jinja2.plugins.tasks import template_file
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
import pynetbox
from nornir_napalm.plugins.tasks import napalm_get
from nornir_napalm.plugins.tasks import napalm_cli
from nornir.core.filter import F
#from nornir.core.inventory import Host
#import json

Инициализируем nornir

In [173]:
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.226:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

Инициализируем pynetbox

In [174]:
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [175]:
device_type = nb.dcim.device_types.get(slug='cisco_ios').id
device_type

""" devices = nb.dcim.devices.filter(
    role='access_switch',
    site = 'plgn',
    device_type = 'cisco_ios'
) """

sw = nr.filter(
    F(site__slug='plgn') & 
    F(device_role__slug='access_switch') &
    F(device_type__slug='cisco_ios')
    )

#result = sw.run(task=napalm_get, getters=['get_interfaces_ip'])
#result = sw.run(napalm_cli,commands=["sh ip int brief"])
#result = sw.run(netmiko_send_command, command_string="show ip int brief")
#print_result(result)


In [178]:
interface = nb.dcim.interfaces.get(name='',device='sw-plgn-1')
interface
""" for i in interfaces:
    print(i) """

Указываем путь к папке с шаблонами

In [134]:
templates_path = "./templates/"

In [ ]:
def config_switches(task:Task) -> Result:
    # ------------------------------- Configure the interface -------------------------------------------------------------------------------------------------------------------- ----------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_interface_template = task.run(
        name='2 шаг. 1: Получаем шаблон конфигурации интерфейсов',  # mission name
        task=template_file,   # The purpose of the task is to get the configuration template, fixed value
        template='cisco_ios_interface.template',
        path=templates_path
    )

    # Introducing in a specific configuration, configuring the device, pay attention to ".split ('\ n')" Convert configuration to a list
    task.run(
        task=netmiko_send_config,  # The purpose of the task is to complete the configuration, fixed value through Netmiko.
        name='2 шаг. 2: Настройка интерфейсов',
        config_commands=ios_interface_template.result.split('\n'), # Convert template information to a list, then configure it via NetMiko
        cmd_verify=True
    )
    
    # --------------------------------- Configure administrator information ----------------------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_user_template = task.run(
        name='2-1: Получаем шаблон конфигурации пользователей',
        task=template_file,
        template='cisco_ios_user.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='2-2: Настройка пользователей',
        config_commands=ios_user_template.result.split('\n'),
        cmd_verify=True
    ) 
    #-------------DNS-------------
    ios_dns_template = task.run(
        name='3-1: Получаем шаблон конфигурации dns',
        task=template_file,
        template='cisco_ios_dns.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='3-2: Настройка dns',
        config_commands=ios_dns_template.result.split('\n'),
        cmd_verify=True
    ) 
    #-------------NTP-------------
    ios_ntp_template = task.run(
        name='4-1: Получаем шаблон конфигурации ntp',
        task=template_file,
        template='cisco_ios_ntp.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='4-2: Настройка ntp',
        config_commands=ios_ntp_template.result.split('\n'),
        cmd_verify=True
    )
   #-------------LOG-------------
    ios_logging_template = task.run(
        name='5-1: Получаем шаблон конфигурации logging',
        task=template_file,
        template='cisco_ios_logging.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='5-2: Настройка syslog',
        config_commands=ios_logging_template.result.split('\n'),
        cmd_verify=True
    )
    #-------------END-------------
    return Result(
        host=task.host,
        result=f"{task.host} доступен"
    )

In [135]:
run_result = sw.run(task=config_switches,name='Настройка коммутаторов')
print_result(run_result)

Настройка коммутаторов**********************************************************
* SW-PLGN-1 ** changed : False *************************************************
vvvv Настройка коммутаторов ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Subtask: 2 шаг. 1: Получаем шаблон конфигурации интерфейсов (failed)

---- 2 шаг. 1: Получаем шаблон конфигурации интерфейсов ** changed : False ----- ERROR
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/nornir/core/task.py", line 99, in start
    r = self.task(self, **self.params)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/nornir_jinja2/plugins/tasks/template_file.py", line 43, in template_file
    text = t.render(host=task.host, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/jinja2/environment.py", line 1301, in render
    self.environment.handle_exception()
  File "/Library/F